In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl
import re

In [2]:
#  Import and read the input csv
wbdi_df = pd.read_csv('../../Raw_Data/World_Bank-world_development_indicators_extract-1990_2022.csv')
wbdi_df

,Country,Country Code,Series Name,Series Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,AFG,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,AFG,"Current health expenditure per capita, PPP (cu...",SH.XPD.CHEX.PP.CD,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,AFG,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,ZWE,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,59.426,58.091,56.435,54.426,52.588,50.531,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,ZWE,Population density (people per sq. km of land ...,EN.POP.DNST,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,10113893,10377815,10641501,10794918,10858594,10994041,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,ZWE,Prevalence of current tobacco use (% of adults),SH.PRV.SMOK,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [3]:
# Drop the extra columns
wbdi_df = wbdi_df.drop(['Country Code','Series Code'], axis=1)

In [4]:
wbdi_df

,Country,Series Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [5]:
pattern = r'\[.*\]'

# Remove the bracketed year info from the column names
wbdi_df.columns=[re.sub(pattern,'',col)for col in wbdi_df.columns]
wbdi_df.columns=wbdi_df.columns.str.strip()
wbdi_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [6]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [7]:
# Apply the correction dictionary to fix the known errors
wbdi_df['Country'] = wbdi_df['Country'].replace(correction_dict)

In [8]:
wbdi_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,Access to electricity (% of population),..,..,..,..,..,..,..,..,...,68.0408782958984,89.5,71.5,97.7,97.7,93.4308776855469,97.7,97.7,97.7,..
1,Afghanistan,Average precipitation in depth (mm per year),327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,..,..
2,Afghanistan,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,8.80596447,9.52887821,10.10534763,11.81859016,12.62081718,14.20841891,14.8313194,15.53361392,16.82606226,..
3,Afghanistan,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,178.07901916923,202.261555308607,215.725233301209,239.188747458965,264.543988949921,299.822543673828,321.525376737099,322.525614744969,..,..
4,Afghanistan,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,0.44335932,0.47265697,0.52267277,0.60012972,0.64313394,0.55239552,0.49705607,1.18621242,0.55571794,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [9]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [10]:
master_countries = countries_df['country'].tolist()
master_countries

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong (China)',
 

In [11]:
# Create a list of wbdi countries
ps_countries = wbdi_df['Country'].tolist()
ps_countries

['Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andor

In [12]:
# clean the FS data based on the SS country list
no_match = []

for country in ps_countries:
    if country in master_countries:
        continue
    else:
        no_match.append(country)

In [13]:
# Likely need to do manual country name clean-up for accented characters and abbreviations.
no_match

['Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Cayman Islands',
 'Cayman Islands',
 'Cayman Isla

In [14]:
wbdi_clean_df = wbdi_df.drop(wbdi_df[wbdi_df['Country'].isin(no_match)].index.tolist())
wbdi_clean_df

,Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
15,Albania,Access to electricity (% of population),100,100,100,100,100,100,100,100,...,99.8903121948242,99.95,99.98,99.89,99.89,100,100,100,100,..
16,Albania,Average precipitation in depth (mm per year),1485,1485,1485,1485,1485,1485,1485,1485,...,1485,1485,1485,1485,1485,1485,1485,1485,..,..
17,Albania,Current health expenditure (% of GDP),..,..,..,..,..,..,..,..,...,6.27234411,6.44164276,6.48361206,6.66606188,6.54604626,6.65958786,..,..,..,..
18,Albania,"Current health expenditure per capita, PPP (cu...",..,..,..,..,..,..,..,..,...,664.902444642237,726.53974963245,755.452047153836,803.796389477951,834.280774470738,895.594815963717,..,..,..,..
19,Albania,Domestic general government health expenditure...,..,..,..,..,..,..,..,..,...,2.57775068,2.5936265,2.731354,2.80370712,2.6750803,2.82325125,2.86730385,2.93465567,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,Zimbabwe,"Life expectancy at birth, total (years)",59.426,58.091,56.435,54.426,52.588,50.531,48.955,47.993,...,57.458,58.846,59.591,60.306,60.709,61.414,61.292,61.124,59.253,..
3251,Zimbabwe,Population density (people per sq. km of land ...,26.144223859377,26.8264572831847,27.5080806514153,27.9046607212098,28.0692619878506,28.4193899444229,28.8953625436216,29.3715936409461,...,35.0405118262893,35.8168618327517,36.5902468657102,37.3599689802249,38.1313196329327,38.9096135453018,39.6913739175391,40.5057929430012,..,..
3252,Zimbabwe,"Population, total",10113893,10377815,10641501,10794918,10858594,10994041,11178171,11362401,...,13555422,13855753,14154937,14452704,14751101,15052184,15354608,15669666,15993524,16320537
3253,Zimbabwe,Prevalence of current tobacco use (% of adults),..,..,..,..,..,..,..,..,...,..,..,13.6,..,..,12.1,12.1,11.7,..,..


In [15]:
# Open the list of series name updates
series_updates_df = pd.read_csv('../../Clean_Data/master_country_list/series_name_updates.csv')

# Convert the series_updates dataframe to a dictionary.
series_updates_dict = dict(zip(series_updates_df['current'], series_updates_df['updated']))

In [16]:
# Apply the series_updates dictionary to reduce verbosity
wbdi_clean_df['Series Name'] = wbdi_clean_df['Series Name'].replace(series_updates_dict)

In [17]:
# Get a list of the columns to convert to numeric
cols = wbdi_clean_df.columns.drop(['Country', 'Series Name'])

wbdi_clean_df[cols] = wbdi_clean_df[cols].apply(pd.to_numeric, errors='coerce')

In [18]:
# Collect the list of data countries - used for cross-data-set analysis of the included countries and whether they have data
data_countries = wbdi_clean_df['Country'].unique()
wbd_countries_df = pd.DataFrame(data_countries, columns=['Country'])
wbd_countries_df.to_csv('../../Clean_Data/Clean_CSV_Files/WBD_countries.csv', index=False)

### Manage missing values

In [19]:
# Create functions to easily repeat the processing, if multiple DataFrames.

def column_names(df):
    # Combine "Country" and "Series Name" as new column to rename
    new_column = (pl.col("Country") + "_" + pl.col("Series Name")).alias("Country_Series")

    # Add new column and place it as the first column  
    df = df.with_columns([new_column]).select([new_column] + df.columns)

    # Drop "Country" and "Series Name" columns
    df = df.drop(['Country', 'Series Name'])
    return df

def df_datatypes(df):
    # Jump through a hoop to set the correct datatype for the value columns after transposition.
    # Convert DataFrame to pandas
    df_pd = df.to_pandas()

    # Set the first row as column headers
    df_pd.columns = df_pd.iloc[0]
    df_pd = df_pd[1:]

    # Convert columns to numeric, coerce non-numeric values to NaN
    for col in df_pd.columns[0:]:
        df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce')

    # Convert back to Polars DataFrame
    df = pl.from_pandas(df_pd)
    return df

def missing_values(df):
    # Fill the missing values
    df = df.interpolate()
    df = df.fill_null(strategy='backward')
    df = df.fill_null(strategy='forward')
    df = df.fill_null(strategy='one')
    return df

In [20]:
# Create a Polars dataframe
wbdi_pl_df = pl.DataFrame(wbdi_clean_df)
wbdi_pl_df.head(10)

Country,Series Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albania""","""Electricity Ac…",100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.430855,99.421989,99.404579,99.385628,99.372139,99.371124,99.389366,99.428452,100.0,99.554451,99.644203,99.718643,99.9,99.890312,99.95,99.98,99.89,99.89,100.0,100.0,100.0,100.0,null
"""Albania""","""Avg. Rainfall …",1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,1485.0,null,null
"""Albania""","""Current HE (% …",null,null,null,null,null,null,null,null,null,null,5.944186,5.925833,5.6645999,6.198919,6.0557785,5.83303,5.6859889,5.708428,5.509624,5.7276206,6.238996,6.119528,6.092626,6.272344,6.441643,6.483612,6.666062,6.546046,6.659588,null,null,null,null
"""Albania""","""Current HE per…",null,null,null,null,null,null,null,null,null,null,222.786533,247.234911,257.917555,304.600289,324.861823,339.739179,372.026996,414.916862,452.67909,504.256292,600.657527,625.645152,643.145746,664.902445,726.53975,755.452047,803.796389,834.280774,895.594816,null,null,null,null
"""Albania""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,2.765835,2.753284,2.588679,2.670353,2.820827,2.665318,2.559311,2.435714,2.2712636,2.338068,2.378746,2.365557,2.455187,2.577751,2.5936265,2.731354,2.803707,2.6750803,2.823251,2.867304,2.934656,null,null
"""Albania""","""Domestic gener…",null,null,null,null,null,null,null,null,null,null,103.662764,114.871264,117.86635,131.214883,151.323064,155.238838,167.452437,177.039776,186.610458,205.842089,229.013045,241.848567,259.172784,273.255533,292.529819,318.249602,338.072137,340.933759,379.676537,398.656673,396.009213,null,null
"""Albania""","""Domestic priva…",null,null,null,null,null,null,null,null,null,null,115.435286,121.4623,131.165703,144.993751,152.854839,164.578373,180.733703,212.806591,254.547993,290.027701,360.162233,378.376081,378.462993,384.546879,429.535609,431.903524,458.6215,480.192965,506.436432,null,null,null,null
"""Albania""","""GDP growth per…",-11.187905,-27.566821,-6.622551,10.229949,8.969762,14.024496,9.78018,-10.361105,9.516484,13.608069,7.630022,9.314397,4.850475,5.924212,5.95657,6.067882,6.572923,6.787229,8.328108,4.05314,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,3.480293,3.898112,4.276312,2.523541,-2.745239,9.922714,6.205275
"""Albania""","""GDP per capita…",2549.746801,1909.31916,1823.503609,2057.692048,2290.143917,2666.081951,2980.418871,2717.684701,3021.179604,3471.646616,3861.333006,4300.222735,4661.402695,5000.295651,5427.767129,5865.291354,6565.633374,7284.166538,8228.354576,8813.51676,9628.640112,10207.759455,10526.250106,10570.966045,11259.303681,11658.894897,12078.799334,12770.991863,13498.184607,14407.437645,14064.038615,15532.706056,18551.716479


In [21]:
# Process the DataFrame
wbdi_pl_df = column_names(wbdi_pl_df)

# Transpose the DataFrame
wbdi_pl_df = wbdi_pl_df.transpose(include_header=True)

# Correct the DataFrame datatypes
wbdi_pl_df = df_datatypes(wbdi_pl_df)

# Filling missing values
wbdi_pl_df = missing_values(wbdi_pl_df)

# The end result
wbdi_pl_df.head(40)

Country_Series,Albania_Electricity Access (% Pop),Albania_Avg. Rainfall (mm/yr),Albania_Current HE (% GDP),Albania_Current HE per capita (PPP Int'l $),Albania_Domestic general gov't HE (% GDP),Albania_Domestic general gov't HE per capita (PPP Int'l $),Albania_Domestic private HE per capita (PPP Int'l $),Albania_GDP growth per capita %,Albania_GDP per capita (PPP Int'l $),Albania_Total Labor Force,Albania_LEx years,Albania_Population Density (people per sq. km),Albania_Total Population,Albania_Tobacco use (% adults),Albania_Alcohol Consumption per capita (liters),Algeria_Electricity Access (% Pop),Algeria_Avg. Rainfall (mm/yr),Algeria_Current HE (% GDP),Algeria_Current HE per capita (PPP Int'l $),Algeria_Domestic general gov't HE (% GDP),Algeria_Domestic general gov't HE per capita (PPP Int'l $),Algeria_Domestic private HE per capita (PPP Int'l $),Algeria_GDP growth per capita %,Algeria_GDP per capita (PPP Int'l $),Algeria_Total Labor Force,Algeria_LEx years,Algeria_Population Density (people per sq. km),Algeria_Total Population,Algeria_Tobacco use (% adults),Algeria_Alcohol Consumption per capita (liters),Andorra_Electricity Access (% Pop),Andorra_Avg. Rainfall (mm/yr),Andorra_Current HE (% GDP),Andorra_Current HE per capita (PPP Int'l $),Andorra_Domestic general gov't HE (% GDP),Andorra_Domestic general gov't HE per capita (PPP Int'l $),…,Yemen_GDP per capita (PPP Int'l $),Yemen_Total Labor Force,Yemen_LEx years,Yemen_Population Density (people per sq. km),Yemen_Total Population,Yemen_Tobacco use (% adults),Yemen_Alcohol Consumption per capita (liters),Zambia_Electricity Access (% Pop),Zambia_Avg. Rainfall (mm/yr),Zambia_Current HE (% GDP),Zambia_Current HE per capita (PPP Int'l $),Zambia_Domestic general gov't HE (% GDP),Zambia_Domestic general gov't HE per capita (PPP Int'l $),Zambia_Domestic private HE per capita (PPP Int'l $),Zambia_GDP growth per capita %,Zambia_GDP per capita (PPP Int'l $),Zambia_Total Labor Force,Zambia_LEx years,Zambia_Population Density (people per sq. km),Zambia_Total Population,Zambia_Tobacco use (% adults),Zambia_Alcohol Consumption per capita (liters),Zimbabwe_Electricity Access (% Pop),Zimbabwe_Avg. Rainfall (mm/yr),Zimbabwe_Current HE (% GDP),Zimbabwe_Current HE per capita (PPP Int'l $),Zimbabwe_Domestic general gov't HE (% GDP),Zimbabwe_Domestic general gov't HE per capita (PPP Int'l $),Zimbabwe_Domestic private HE per capita (PPP Int'l $),Zimbabwe_GDP growth per capita %,Zimbabwe_GDP per capita (PPP Int'l $),Zimbabwe_Total Labor Force,Zimbabwe_LEx years,Zimbabwe_Population Density (people per sq. km),Zimbabwe_Total Population,Zimbabwe_Tobacco use (% adults),Zimbabwe_Alcohol Consumption per capita (liters)
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,100.0,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,-11.187905,2549.746801,1.374478e6,73.144,119.946788,3.286542e6,35.0,6.57,98.64003,89.0,3.489033,297.265963,2.510828,213.922798,83.188034,-1.682928,6974.076379,6.367101e6,67.416,10.714047,2.5518074e7,22.2,0.58,100.0,1.0,5.952103,1913.36792,3.900492,1253.855476,…,1.0,2.905158e6,58.699,25.333108,1.3375121e7,27.3,0.79,13.9,1020.0,7.152371,123.893055,3.18158,55.111193,65.070345,-3.007447,1541.301115,2.335479e6,47.926,10.339662,7.686401e6,19.9,4.05,28.2,657.0,10.475839,189.678186,2.753357,49.852982,87.708421,4.15834,1794.153646,3.516593e6,59.426,26.144224,1.0113893e7,21.0,2.46
1991,100.0,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,-27.566821,1909.31916,1.429833e6,73.378,119.225912,3.26679e6,35.0,6.57,98.64003,89.0,3.489033,297.265963,2.510828,213.922798,83.188034,-3.528168,6955.544827,6.625804e6,67.688,10.97261,2.6133905e7,22.2,0.58,100.0,1.0,5.952103,1913.36792,3.900492,1253.855476,…,1.0,3.009429e6,59.049,26.319395,1.38958

In [22]:
# Convert the Polars DataFrame to Pandas

df = wbdi_pl_df.to_pandas()

# Rename the Country_Series column to "Year"
df.rename(columns={"Country_Series": "Year"}, inplace=True)

# Melt the DataFrame from Wide to Long format
df = df.melt(id_vars='Year', var_name='Country_Series', value_name='Value')

# Separate the Country from the Series Name
df[['Country', 'Series']] = df['Country_Series'].str.rsplit(pat='_', n=1, expand=True)

# Create a "country_year" column
df['Country_Year'] = df['Country'] + '_' + df['Year'].astype(str)

# Pivot the DataFrame to create tidy data
df_final = df.pivot(index='Country_Year', columns='Series', values='Value')
df_final.reset_index(inplace=True)

df_final

Series,Country_Year,Alcohol Consumption per capita (liters),Avg. Rainfall (mm/yr),Current HE (% GDP),Current HE per capita (PPP Int'l $),Domestic general gov't HE (% GDP),Domestic general gov't HE per capita (PPP Int'l $),Domestic private HE per capita (PPP Int'l $),Electricity Access (% Pop),GDP growth per capita %,GDP per capita (PPP Int'l $),LEx years,Population Density (people per sq. km),Tobacco use (% adults),Total Labor Force,Total Population
0,Albania_1990,6.57,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,100.000000,-11.187905,2549.746801,73.144,119.946788,35.0,1374478.0,3286542.0
1,Albania_1991,6.57,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,100.000000,-27.566821,1909.319160,73.378,119.225912,35.0,1429833.0,3266790.0
2,Albania_1992,6.57,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,100.000000,-6.622551,1823.503609,73.715,118.505073,35.0,1438342.0,3247039.0
3,Albania_1993,6.57,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,100.000000,10.229949,2057.692048,73.939,117.784197,35.0,1413557.0,3227287.0
4,Albania_1994,6.57,1485.0,5.944186,222.786533,2.765835,103.662764,115.435286,100.000000,8.969762,2290.143917,74.131,117.063358,35.0,1391914.0,3207536.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6034,Zimbabwe_2018,4.67,657.0,4.670418,180.091128,1.594279,61.475349,64.993823,45.400288,2.909395,2457.309859,61.414,38.909614,12.1,5770226.0,15052184.0
6035,Zimbabwe_2019,4.67,657.0,3.659765,132.068691,0.522077,18.839995,31.431890,46.682095,-8.177320,2296.845429,61.292,39.691374,12.1,5912685.0,15354608.0
6036,Zimbabwe_2020,4.67,657.0,3.425581,118.127063,0.756783,26.096745,26.263578,52.747667,-9.670405,2101.804597,61.124,40.505793,11.7,6008633.0,15669666.0
6037,Zimbabwe_2021,4.67,657.0,3.425581,118.127063,0.756783,26.096745,26.263578,48.979927,6.271613,2333.973632,59.253,40.505793,11.7,6200666.0,15993524.0


In [23]:
#wbdi_pl_df.write_csv('../../Clean_Data/Clean_CSV_Files/world_bank_dev_indicators.csv', has_header=True, quote_style='necessary')

In [24]:
df_final.to_csv('../../Clean_Data/Clean_CSV_Files/world_bank_dev_indicators.csv', index=False)

In [25]:
#fsle_clean_transform_df = fsle_clean_df.set_index('Entity').T
#fsle_clean_transform_df

In [26]:
#fsle_clean_transform_df.to_csv('food_supply_life_expectancy_transform.csv')